## Step 1: Installation & Setup

In [ ]:
# Install dependencies (uncomment if needed)
# !pip install langchain-zendfi langchain-openai python-dotenv

In [ ]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Set your API keys (or use .env file)
# os.environ["ZENDFI_API_KEY"] = "zk_test_your_key_here"
# os.environ["OPENAI_API_KEY"] = "sk-your_openai_key_here"

# Verify setup
assert os.getenv("ZENDFI_API_KEY"), "Please set ZENDFI_API_KEY"
assert os.getenv("OPENAI_API_KEY"), "Please set OPENAI_API_KEY"
print("✅ API keys configured!")

## Step 2: Import ZendFi Tools

ZendFi provides four LangChain tools:
- `ZendFiPaymentTool`: Make cryptocurrency payments
- `ZendFiBalanceTool`: Check your balance and limits
- `ZendFiMarketplaceTool`: Search for service providers
- `ZendFiCreateSessionTool`: Create new session keys

In [ ]:
from langchain_zendfi import (
    ZendFiPaymentTool,
    ZendFiBalanceTool,
    ZendFiMarketplaceTool,
    ZendFiCreateSessionTool,
    create_zendfi_tools,
)

print("✅ ZendFi tools imported!")

## Step 3: Create Tools with Configuration

You can create tools individually or use `create_zendfi_tools()` for convenience.

In [ ]:
# Option 1: Create all tools at once (recommended)
tools = create_zendfi_tools(
    mode="test",           # 'test' for devnet, 'live' for mainnet
    session_limit_usd=10.0, # $10 spending limit
    debug=True,             # Enable logging
)

print(f"Created {len(tools)} tools:")
for tool in tools:
    print(f"  • {tool.name}: {tool.description[:60]}...")

In [ ]:
# Option 2: Create tools individually for custom configuration
payment_tool = ZendFiPaymentTool(
    mode="test",
    session_limit_usd=5.0,  # Different limit
)

balance_tool = ZendFiBalanceTool(
    mode="test",
    session_limit_usd=5.0,
)

print(f"Payment tool: {payment_tool.name}")
print(f"Balance tool: {balance_tool.name}")

## Step 4: Test Tools Directly

Before using with an agent, let's test the tools directly.

In [ ]:
# Test: Check balance
result = balance_tool.invoke({})
print(result)

In [ ]:
# Test: Search marketplace for GPT-4 token providers
marketplace_tool = ZendFiMarketplaceTool(mode="test")

result = marketplace_tool.invoke({
    "service_type": "gpt4-tokens",
    "max_price": 0.15,
    "min_reputation": 4.0,
})
print(result)

In [ ]:
# Test: Make a payment (using mock provider from marketplace)
result = payment_tool.invoke({
    "recipient": "BetaProvider1234567890abcdef",
    "amount_usd": 0.60,
    "description": "10 GPT-4 tokens at $0.06/token",
})
print(result)

## Step 5: Create a LangChain Agent

Now let's give these tools to a GPT-4 agent!

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate

# Create the LLM
llm = ChatOpenAI(model="gpt-4o", temperature=0)

# Create prompt
prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a helpful AI assistant that can make cryptocurrency 
payments on Solana. You have tools to:
- Check your payment balance
- Search for service providers in the marketplace
- Make payments to other wallets

Always check your balance before making payments.
Be clear about what you're doing and why."""),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}"),
])

# Create agent
agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
)

print("✅ Agent created!")

## Step 6: Talk to Your Agent!

Now you can interact with your agent naturally.

In [ ]:
# Ask about balance
response = agent_executor.invoke({
    "input": "What's my current payment balance?"
})
print("\n🤖 Agent says:")
print(response["output"])

In [ ]:
# Search for providers
response = agent_executor.invoke({
    "input": "Find me providers that sell GPT-4 tokens for under $0.10 each"
})
print("\n🤖 Agent says:")
print(response["output"])

In [ ]:
# Make an autonomous purchase!
response = agent_executor.invoke({
    "input": """Please buy 10 GPT-4 tokens from the cheapest provider 
    that has at least 4.0 rating. My budget is $1.00."""
})
print("\n🤖 Agent says:")
print(response["output"])

## Step 7: Advanced - Full Autonomous Commerce

Watch the agent handle a complex multi-step purchase autonomously.

In [ ]:
# Complex autonomous task
autonomous_task = """
I need to complete two purchases:

1. First, buy 5 GPT-4 tokens from the best-priced provider with 4.5+ rating
2. Then, if I have at least $0.50 remaining, also buy some image generation 
   credits from a provider with 4.0+ rating

For each purchase:
- Find the best provider meeting my criteria
- Execute the payment
- Confirm success

At the end, tell me my final balance and what I purchased.
"""

response = agent_executor.invoke({"input": autonomous_task})
print("\n🤖 Agent says:")
print(response["output"])

## 🎉 Congratulations!

You've successfully:
- Set up ZendFi tools for LangChain
- Created an agent with payment capabilities
- Made autonomous cryptocurrency payments
- Searched the marketplace for providers

### What's Next?

- **Production Mode**: Change `mode="live"` for mainnet
- **Higher Limits**: Increase `session_limit_usd` for larger budgets
- **Custom Agents**: Create specialized agents for specific tasks
- **Multi-Agent**: Build systems where agents trade with each other

### Resources

- [ZendFi Documentation](https://docs.zendfi.com)
- [LangChain ZendFi Guide](https://docs.zendfi.com/langchain)
- [GitHub Repository](https://github.com/zendfi/langchain-zendfi)